In [1]:
from sqlalchemy import create_engine, text

In [2]:
#Creating engine

username = 'postgres'
password = 'hellosql'
database = 'DTW Flights'
host = '127.0.0.1'
port = '5432'  # Default PostgreSQL port

# Create the connection string
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Create the engine
engine = create_engine(connection_string)

## Read both tables using only SQLAlchemy (no Pandas). Use a separate command for each.

In [8]:
#read tables from database
with engine.connect() as conn:
    flights = text("Select * from flights")
    resultset_flights = conn.execute(flights)

In [9]:
flights_as_dict = resultset_flights.mappings().all()
print(flights_as_dict)

[{'id': 1, 'origin': 'DTW', 'destination': 'ORD', 'date': datetime.date(2024, 1, 14), 'status': 'Delayed'}, {'id': 2, 'origin': 'LAX', 'destination': 'DTW', 'date': datetime.date(2024, 1, 15), 'status': 'On Time'}, {'id': 3, 'origin': 'DTW', 'destination': 'ATL', 'date': datetime.date(2024, 1, 16), 'status': 'Delayed'}, {'id': 4, 'origin': 'SEA', 'destination': 'DTW', 'date': datetime.date(2024, 1, 17), 'status': 'On Time'}, {'id': 5, 'origin': 'JFK', 'destination': 'DTW', 'date': datetime.date(2024, 1, 18), 'status': 'Canceled'}, {'id': 6, 'origin': 'DTW', 'destination': 'DEN', 'date': datetime.date(2024, 1, 19), 'status': 'Delayed'}, {'id': 7, 'origin': 'DTW', 'destination': 'PHX', 'date': datetime.date(2024, 1, 20), 'status': 'On Time'}, {'id': 8, 'origin': 'DTW', 'destination': 'BOS', 'date': datetime.date(2024, 1, 21), 'status': 'On Time'}, {'id': 9, 'origin': 'DTW', 'destination': 'IAD', 'date': datetime.date(2024, 1, 22), 'status': 'Canceled'}, {'id': 10, 'origin': 'DCA', 'desti

In [10]:
with engine.connect() as conn:
    bookings = text("Select * from bookings")
    resultset_bookings = conn.execute(bookings)

In [11]:
bookings_as_dict = resultset_bookings.mappings().all()
print(bookings_as_dict)

[{'id': 1, 'name': 'Elena Rodriguez', 'seat': '22F', 'flight_id': 3}, {'id': 2, 'name': 'Emma Johnson', 'seat': '7C', 'flight_id': 2}, {'id': 3, 'name': 'Michael Davis', 'seat': '12B', 'flight_id': 7}, {'id': 4, 'name': 'Sophia Wilson', 'seat': '18D', 'flight_id': 5}, {'id': 5, 'name': 'Daniel Martinez', 'seat': '24E', 'flight_id': 3}, {'id': 6, 'name': 'Olivia Taylor', 'seat': '10F', 'flight_id': 7}, {'id': 7, 'name': 'Matthew Brown', 'seat': '5A', 'flight_id': 3}, {'id': 8, 'name': 'Isabella Lee', 'seat': '20B', 'flight_id': 7}, {'id': 9, 'name': 'Ethan Anderson', 'seat': '14C', 'flight_id': 9}, {'id': 10, 'name': 'Ava Garcia', 'seat': '8D', 'flight_id': 10}]


## Create a Pandas Dataframe for each table and print them.


In [14]:
import pandas as pd
with engine.connect() as conn:
    flights = text("Select * from flights")
    resultset_flights = conn.execute(flights)
    flights_df = pd.DataFrame(resultset_flights.fetchall(), columns=resultset_flights.keys())
    
flights_df

,id,origin,destination,date,status
0,1,DTW,ORD,2024-01-14,Delayed
1,2,LAX,DTW,2024-01-15,On Time
2,3,DTW,ATL,2024-01-16,Delayed
3,4,SEA,DTW,2024-01-17,On Time
4,5,JFK,DTW,2024-01-18,Canceled
5,6,DTW,DEN,2024-01-19,Delayed
6,7,DTW,PHX,2024-01-20,On Time
7,8,DTW,BOS,2024-01-21,On Time
8,9,DTW,IAD,2024-01-22,Canceled
9,10,DCA,DTW,2024-01-23,On Time


In [15]:
with engine.connect() as conn:
    bookings = text("Select * from bookings")
    resultset_bookings = conn.execute(bookings)
    bookings_df = pd.DataFrame(resultset_bookings.fetchall(), columns=resultset_bookings.keys())
    
bookings_df

,id,name,seat,flight_id
0,1,Elena Rodriguez,22F,3
1,2,Emma Johnson,7C,2
2,3,Michael Davis,12B,7
3,4,Sophia Wilson,18D,5
4,5,Daniel Martinez,24E,3
5,6,Olivia Taylor,10F,7
6,7,Matthew Brown,5A,3
7,8,Isabella Lee,20B,7
8,9,Ethan Anderson,14C,9
9,10,Ava Garcia,8D,10


## Use an SQL JOIN based on the flight id to load both tables into a single Pandas Dataframe.

In [22]:
flight_id_sql = """SELECT b.*, f.origin, f.destination, f.date, f.status
FROM bookings as b
LEFT JOIN flights as f ON f.id = b.flight_id"""
flight_id_df = pd.read_sql(flight_id_sql, engine)

In [23]:
flight_id_df

,id,name,seat,flight_id,origin,destination,date,status
0,1,Elena Rodriguez,22F,3,DTW,ATL,2024-01-16,Delayed
1,2,Emma Johnson,7C,2,LAX,DTW,2024-01-15,On Time
2,3,Michael Davis,12B,7,DTW,PHX,2024-01-20,On Time
3,4,Sophia Wilson,18D,5,JFK,DTW,2024-01-18,Canceled
4,5,Daniel Martinez,24E,3,DTW,ATL,2024-01-16,Delayed
5,6,Olivia Taylor,10F,7,DTW,PHX,2024-01-20,On Time
6,7,Matthew Brown,5A,3,DTW,ATL,2024-01-16,Delayed
7,8,Isabella Lee,20B,7,DTW,PHX,2024-01-20,On Time
8,9,Ethan Anderson,14C,9,DTW,IAD,2024-01-22,Canceled
9,10,Ava Garcia,8D,10,DCA,DTW,2024-01-23,On Time


## List all flights coming into Detroit on January 18th, 2024. (You may choose whether to do this with an SQL query or using Pandas features.)

In [25]:
dtw_sql = """SELECT *
FROM flights
WHERE date = '2024-01-18' and destination = 'DTW'"""
dtw_df = pd.read_sql(dtw_sql, engine)

dtw_df

,id,origin,destination,date,status
0,5,JFK,DTW,2024-01-18,Canceled
1,15,ATL,DTW,2024-01-18,On Time


## Create a new flight from Detroit to Seattle:(id should be generated automatically by the database)
origin: DTW
destination: SEA
date: 2024-01-10
status: On ime



In [29]:
origin = 'DTW'
destination = 'SEA'
date = '2024-01-10'
status = 'On time'
insert_flights_sql = '''
  INSERT INTO flights (origin, destination, date, status)
  VALUES (:origin, :destination, :date, :status)
'''

In [30]:
with engine.connect() as conn:
    trans=conn.begin()
    new_flight = text(insert_flights_sql)
    resultset = conn.execute(new_flight,  {'origin': origin, 'destination' : destination, 'date' : date, 'status' : status })
    trans.commit()

## Update this new DTW → SEA flight. The status is now delayed

In [37]:
update_sql = """ UPDATE Flights
SET status = 'Delayed'
WHERE date = '2024-01-10' and origin = 'DTW' and destination = 'SEA'"""

In [38]:
with engine.connect() as conn:
    trans=conn.begin()
    update_flight = text(update_sql)
    resultset = conn.execute(update_flight)
    trans.commit()

## Create a new flight from Los Angeles to Detroit

(id should be generated automatically by the database)
origin: LAX
destination: DTW
date: 2024-01-16
status: Delyed


In [39]:
origin = 'LAX'
destination = 'DTW'
date = '2024-01-16'
status = 'Delayed'
insert_flights_sql = '''
  INSERT INTO flights (origin, destination, date, status)
  VALUES (:origin, :destination, :date, :status)
'''

In [40]:
with engine.connect() as conn:
    trans=conn.begin()
    new_flight = text(insert_flights_sql)
    resultset = conn.execute(new_flight,  {'origin': origin, 'destination' : destination, 'date' : date, 'status' : status })
    trans.commit()

## There is a winter storm in the weather forecast. 
First, update all bookings going from LAX to DTW on 01/15/2024 to the flight created in the last prompt..


In [45]:
update_sql = """
UPDATE bookings
SET flight_id = 23
WHERE id = 2 and seat = '7C'
"""

In [46]:
with engine.connect() as conn:
    trans=conn.begin()
    update_flight = text(update_sql)
    resultset = conn.execute(update_flight)
    trans.commit()

Then, delete all flights scheduled 01/15/2024.

In [49]:
del_sql = """
DELETE FROM flights WHERE date = '2024-01-15' """

In [50]:
with engine.connect() as conn:
    trans=conn.begin()
    del_flight = text(del_sql)
    resultset = conn.execute(del_flight)
    trans.commit()